In [6]:
import dspy
import os
from dotenv import load_dotenv
load_dotenv()
load_dotenv('.secrets')

True

In [11]:

class BoardGameResearchPlanner(dspy.Signature):
    question = dspy.InputField(desc="The user's initial query about board games")
    subquestions: list[str] = dspy.OutputField(desc="A list of subquestions that get deeper into the theme the user is making research")

class BoardGameResearcher(dspy.Signature):
    question = dspy.InputField(desc="The user's initial query about board games")
    subquestion: str = dspy.InputField(desc="The subquestion you are doing deep research on")
    context = dspy.OutputField(desc="The context learned by answering the subquestion while relevant to the initial question")

class BoardGameQuestionAnswer(dspy.Signature):
    question = dspy.InputField(desc="The user's initial query about board games")
    answer = dspy.OutputField(desc="The final answer including all the context researched about board games.")

class BoardGameResearchSummarizer(dspy.Signature):
    """Condense retrieved information into a focused summary."""
    question = dspy.InputField(desc="The user's initial query about board games")
    context: list[str] = dspy.InputField(desc="Research context discovered about the board games you are about to recommend")
    summary = dspy.OutputField(desc="Condense retrieved information into a focused summary relevant to the provided context and user question.")


class BoardGameAssistant(dspy.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.plan_research = dspy.ChainOfThought(BoardGameResearchPlanner)
        self.research = dspy.ChainOfThought(BoardGameResearcher)
        self.summarize = dspy.ChainOfThought(BoardGameResearchSummarizer)
        self.answer = dspy.ChainOfThought(BoardGameQuestionAnswer)

    def forward(self, question):
        subquestions = self.plan_research(question=question).subquestions
        research_context = []
        print(f"Question: {question} research tasks: {len(subquestions)}")
        for subquestion in subquestions:
            print(f" - Researching: {subquestion}")
            research = self.research(subquestion=subquestion, question=question)
            research_context = research_context + [research.context]

        final_research = self.summarize(context=research_context, question=question)

        return dspy.Prediction(answer=final_research.summary)

In [12]:
lm = dspy.LM('openai/gpt-4o', api_key=os.getenv("OPENAI_API_KEY"))
dspy.configure(lm=lm)

In [14]:
assistant = BoardGameAssistant()
recommendation = assistant(question="What is the best opening move in Catan?")

In [15]:
recommendation


Prediction(
    answer='The best opening move in Catan involves placing initial settlements on intersections with high-frequency numbers like 6 and 8 to ensure consistent resource production. Players should prioritize access to wood, brick, wheat, and sheep for early expansion. The strategy should adapt to the number of players, with more selectivity in a 3-player game and a balanced approach in a 4-player game. Avoiding common mistakes, such as neglecting resource balance and future growth, is crucial. Strategic placement should also consider long-term trading potential and expansion opportunities.'
)